In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tqdm
import time
import os
import random
import copy
import sys

## Implementation of Refined Partition procedure for general dependency matrices

In [ ]:
import networkx as nx

In [ ]:
def adjacency_matrix_to_bipartite_graph(adj_matrix):
    # Initialize bipartite graph
    B = nx.Graph()

    # Number of functions and variables (square matrix assumption)
    num_functions = adj_matrix.shape[0]
    num_variables = adj_matrix.shape[1]

    # Add function nodes
    function_nodes = [f'f_{i+1}' for i in range(num_functions)]
    B.add_nodes_from(function_nodes, bipartite=0)  # bipartite=0 means these are one set of nodes

    # Add variable nodes
    variable_nodes = [f'w_{i+1}' for i in range(num_variables)]
    B.add_nodes_from(variable_nodes, bipartite=1)  # bipartite=1 means these are the second set of nodes

    # Add edges based on the adjacency matrix
    for i in range(num_functions):
        for j in range(num_variables):
            if adj_matrix[i, j] == 1:
                B.add_edge(f'f_{i+1}', f'w_{j+1}')

    return B, function_nodes, variable_nodes

In [ ]:
def plot_bipartite_graph(B, function_nodes, variable_nodes):
    # Set the positions for the two sets of nodes: functions on top, variables on the bottom
    pos = {}
    pos.update((node, (i, 1)) for i, node in enumerate(function_nodes))  # Function nodes on y=1
    pos.update((node, (i, -1)) for i, node in enumerate(variable_nodes))  # Variable nodes on y=-1

    # Plotting the bipartite graph
    plt.figure(figsize=(8, 6))
    nx.draw(B, pos, with_labels=True, node_color=['lightblue' if node.startswith('f') else 'lightgreen' for node in B.nodes()],
            node_size=2000, font_size=12, font_weight='bold')
    plt.show()

In [ ]:
def detect_cycle(B):
    """
    Detect a cycle in the graph B using NetworkX's cycle detection method.

    Parameters:
    B (networkx.Graph): The input bipartite graph.

    Returns:
    list: A list of edges forming the cycle if found, otherwise an empty list.
    """
    try:
        cycle = nx.find_cycle(B)
        return cycle
    except nx.exception.NetworkXNoCycle:
        print("No cycle found.")
        return []

In [ ]:
def plot_highlight_cycle(B, cycle, function_nodes, variable_nodes):
    """
    Plot the bipartite graph B and highlight the edges in the cycle.
    Arrange function nodes at the top and variable nodes at the bottom.

    Parameters:
    B (networkx.Graph): The input bipartite graph.
    cycle (list): A list of edges forming the cycle to be highlighted.
    function_nodes (list): List of function nodes (e.g., ['f_1', 'f_2', ...]).
    variable_nodes (list): List of variable nodes (e.g., ['w_1', 'w_2', ...]).
    """
    # Set the positions for the two sets of nodes: functions on top, variables on the bottom
    pos = {}
    pos.update((node, (i, 1)) for i, node in enumerate(function_nodes))  # Function nodes on y=1
    pos.update((node, (i, -1)) for i, node in enumerate(variable_nodes))  # Variable nodes on y=-1

    # Edge color red for edges in the cycle, black otherwise
    edge_colors = ['red' if edge in cycle or (edge[1], edge[0]) in cycle else 'black' for edge in B.edges()]

    # Plot the graph
    plt.figure(figsize=(8, 6))
    nx.draw(B, pos, with_labels=True, node_color='lightblue', edge_color=edge_colors, node_size=2000, font_size=12)
    plt.show()

In [ ]:
def merge_cycle_variable_nodes(B, cycle, variable_nodes):
    """
    Merge variable nodes within the detected cycle into a single node in a new graph.

    Parameters:
    B (networkx.Graph): The original bipartite graph.
    cycle (list): A list of edges forming the cycle to be merged.
    variable_nodes (list): List of variable nodes to be considered for merging.

    Returns:
    new_graph (networkx.Graph): A new graph with the cycle variables merged.
    merged_node_name (str): The name of the new merged node.
    merged_variables (list): List of variable nodes that were merged.
    """
    new_graph = B.copy()  # Create a copy of the original graph to modify

    # Extract the variable nodes involved in the cycle
    cycle_variables = set()
    for u, v in cycle:
        if u in variable_nodes:
            cycle_variables.add(u)
        if v in variable_nodes:
            cycle_variables.add(v)

    # If no variable nodes were found in the cycle, return the original graph
    if not cycle_variables:
        print("No variable nodes in the cycle to merge.")
        return new_graph, None, []

    # Create a new node name to represent the merged variables
    merged_node_name = "_".join(sorted(cycle_variables))  # E.g., "w_1_w_2"

    # Add the merged node to the new graph
    new_graph.add_node(merged_node_name)

    # Keep track of the merged variable names
    merged_variables = list(cycle_variables)

    # Find the functions connected to the cycle variables and link them to the merged node
    for var in merged_variables:
        # Find neighbors (functions) of the variable node
        neighbors = list(B.neighbors(var))
        for neighbor in neighbors:
            # Add an edge between the merged node and the function
            new_graph.add_edge(merged_node_name, neighbor)

        # Remove the original variable node from the graph
        new_graph.remove_node(var)

    return new_graph, merged_node_name, merged_variables

In [ ]:
def iterative_cycle_detection_and_merge(B, function_nodes, variable_nodes):
    """
    Repeat the process of detecting and merging cycles in the graph B until no cycle is detected.

    Parameters:
    B (networkx.Graph): The initial bipartite graph.
    function_nodes (list): List of function nodes (e.g., ['f_1', 'f_2', ...]).
    variable_nodes (list): List of variable nodes (e.g., ['w_1', 'w_2', ...]).

    Returns:
    new_graph (networkx.Graph): The final graph after all cycles are merged.
    merged_variable_list (list): A list of all merged variable nodes.
    function_nodes (list): The final list of function nodes.
    variable_nodes (list): The final list of variable nodes, including merged nodes.
    """
    merged_variable_list = []
    merged_count = 0

    # Iteratively detect and merge cycles
    while True:
        # Detect a cycle in the current graph
        cycle = detect_cycle(B)

        # If no cycle is detected, break the loop
        if not cycle:
            break

        # Merge variable nodes involved in the detected cycle
        B, merged_node, merged_vars = merge_cycle_variable_nodes(B, cycle, variable_nodes)

        # If a merge happened, update the variable_nodes list
        if merged_node:
            # Shorten the name of the merged node to avoid long names
            merged_count += 1
            shortened_merged_node = f"merged_{merged_count}"

            # Rename the merged node in the graph
            B = nx.relabel_nodes(B, {merged_node: shortened_merged_node})

            # Update the list of variable nodes by removing merged ones and adding the new one
            variable_nodes = [node for node in variable_nodes if node not in merged_vars]
            variable_nodes.append(shortened_merged_node)

            # Keep track of merged variables for later reference
            merged_variable_list.append((shortened_merged_node, merged_vars))

    # Return the final graph, merged variable list, and updated function/variable nodes
    return B, merged_variable_list, function_nodes, variable_nodes


In [ ]:
def post_process_merged_variables(merged_variable_list):
    """
    Post-process merged variable list to flatten any nested merged variables.

    Parameters:
    merged_variable_list (list): List of tuples where each tuple contains a merged node and its merged variables.

    Returns:
    list: The post-processed list with flattened merged variables.
    """
    # Create a mapping from each merged node to the variables it includes
    merged_dict = {merged_node: set(vars) for merged_node, vars in merged_variable_list}

    # Helper function to recursively gather all original variables
    def get_original_vars(node):
        if node in merged_dict:
            # Recursively flatten merged variables
            result = set()
            for var in merged_dict[node]:
                result.update(get_original_vars(var))
            return result
        else:
            # This is an original variable (w_1, w_2, etc.)
            return {node}

    # Flatten all merged variables
    flattened_list = []
    for merged_node, vars in merged_variable_list:
        original_vars = get_original_vars(merged_node)
        flattened_list.append((merged_node, list(original_vars)))

    return flattened_list

In [ ]:
def filter_existing_merged_nodes(flattened_merged_variable_list, final_graph):
    """
    Filter and print only the merged variables that still exist in the final graph.

    Parameters:
    flattened_merged_variable_list (list): List of flattened merged nodes and their original variables.
    final_graph (networkx.Graph): The final graph after all merging steps.

    Returns:
    list: A filtered list of merged nodes that still exist in the final graph.
    """
    # Get the nodes that exist in the final graph
    existing_nodes = set(final_graph.nodes())

    # Filter the merged variable list to include only the nodes that still exist in the graph
    filtered_list = [(merged_node, vars) for merged_node, vars in flattened_merged_variable_list if merged_node in existing_nodes]

    return filtered_list

In [ ]:
def generate_sparse_adj_matrix(rows, cols, density, seed=None):
    """
    Generates a random sparse adjacency matrix with at least one non-zero entry in each column
    and ensures that the num_non_zero 1s are assigned exactly to empty entries.

    Parameters:
    rows (int): Number of rows.
    cols (int): Number of columns.
    density (float): Approximate density of non-zero elements (between 0 and 1).
    seed (int, optional): Random seed for reproducibility. Defaults to None.

    Returns:
    numpy.ndarray: The generated sparse adjacency matrix.
    """

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    adj_matrix = np.zeros((rows, cols), dtype=int)

    # Ensure at least one non-zero entry in each column
    for j in range(cols):
        adj_matrix[random.randint(0, rows - 1), j] = 1

    # Calculate the number of additional non-zero entries needed
    num_elements = rows * cols
    num_non_zero = int(density * num_elements)
    additional_non_zero = num_non_zero - cols  # Subtract cols as we already have one per column

    # Find indices of empty entries
    empty_indices = np.where(adj_matrix == 0)
    num_empty_entries = len(empty_indices[0])

    # If there are enough empty entries, assign 1s to them
    if additional_non_zero <= num_empty_entries:
        # Randomly select indices to assign 1s
        selected_indices = random.sample(range(num_empty_entries), additional_non_zero)

        # Assign 1s to the selected empty entries
        for index in selected_indices:
            row = empty_indices[0][index]
            col = empty_indices[1][index]
            adj_matrix[row, col] = 1
    else:
        # If there are not enough empty entries, fill all remaining empty entries with 1s
        adj_matrix[empty_indices] = 1

    return adj_matrix

# The impact of adjacency matrix density on variable merging

In [ ]:
# Experiment parameters
rows = 100
cols = 100
densities = [0.015, 0.018, 0.02, 0.025, 0.03, 0.04, 0.05, 0.06, 0.07, 0.1]
num_trials = 100

# Store results
results = {density: [] for density in densities}

# Run the experiment
for density in densities:
    for trial in range(num_trials):
        # Generate sparse adjacency matrix with a random seed
        seed = trial  # Use trial number as seed for different random matrices
        adj_matrix = generate_sparse_adj_matrix(rows, cols, density, seed=seed)

        # Create bipartite graph
        B, function_nodes, variable_nodes = adjacency_matrix_to_bipartite_graph(adj_matrix)

        # Run iterative cycle detection and merging
        final_graph, merged_variable_list, function_nodes, variable_nodes = iterative_cycle_detection_and_merge(B, function_nodes, variable_nodes)

        # Record the number of variable nodes in the final graph
        results[density].append(len(variable_nodes))  # Store results as len(variable_nodes)

# Print results
for density, values in results.items():
    # Calculate and print the average number of variable nodes
    avg_variable_nodes = np.mean(values)  # Directly calculate the mean of the values list
    print(f"Density: {density}, Average variable nodes: {avg_variable_nodes}")

In [ ]:
import pickle

# Store results in a pickle file
filename = 'sparsity_results_dict.pickle'
with open(filename, 'wb') as file:
    pickle.dump(results, file)

### Plots

In [ ]:
with open('sparsity_results_dict.pickle', 'rb') as file:
    results = pickle.load(file)

In [ ]:
# Plotting the box plot
plt.figure(figsize=(10, 6))

# Prepare data for box plot
data = [np.sqrt(values) for values in results.values()]  # Apply square root transformation
labels = ['{:.1f}'.format(density * 100) for density in results.keys()]

# Exclude the last density
data = data[:-1]  # Slice data to exclude the last element
labels = labels[:-1]  # Slice labels to exclude the last element

medianprops = dict(color='orange', linewidth=2)
whiskerprops = dict(color='black')
capprops = dict(color='black', linewidth=1.5)
plt.boxplot(data, labels=labels, showfliers=False, showmeans=False, medianprops = medianprops,
            whiskerprops=whiskerprops, capprops=capprops)  # Create box plot

# Increase tick label font size
plt.tick_params(axis='both', which='major', labelsize=12)
plt.xlabel("Density (%)",fontsize=17)
plt.ylabel("# of Variable Blocks (Square root)",fontsize=17)  # Update y-axis label
plt.title("Effect of Sparsity on Variable Merging",fontsize=20)
plt.grid(True)
# Add y-tick at y=1 and remove y-tick at y=12
yticks = list(plt.yticks()[0])  # Get current y-ticks
yticks = [tick for tick in yticks if tick != 12]  # Remove y-tick at y=12
yticks.append(1)  # Add y-tick at y=1
plt.yticks(yticks)

plt.savefig('boxplot_sparsity_bigger_font.png', dpi=300)
# plt.show()

# MGDA vs RP-MGDA on quadratic MOO problem

## MGDA helper functions

In [ ]:
!pip install quadprog

In [ ]:
import quadprog

In [ ]:
# QP package to solve for the min norm element
def solve_w(U):
    n,d = U.shape
    K = np.eye(n,dtype=float)
    for i in range(0,n):
        for j in range(0,n):
            K[i,j]= np.dot(U[i],U[j])
    Q = 0.5 *(K + K.T)
    p = np.zeros(n,dtype=float)
    a = np.ones(n,dtype=float).reshape(-1,1)
    Id = np.eye(n,dtype=float)
    A = np.concatenate((a,Id),axis=1)
    b = np.zeros(n+1)
    b[0] = 1.
    grad = np.zeros(d,dtype=float)
    # grad = np.zeros(n,dtype=float)
    try:
        grad = U.T.dot(quadprog.solve_qp(Q,p,A,b)[0])
    except ValueError as v:
        print('MGDA stops since the min norm element is zero')
    return grad

In [ ]:
# a FW procedure to solve for the min norm element
# step_size (sz=2/(t+2))
def FW_solve_w(U, rounds):
  n,d = U.shape
  lbd = 1.0/n * np.ones(n)
  G = np.array(U.dot(U.T))
  for t in range(rounds):
    v = G.dot(lbd)
    idx_min = np.argmin(v)
    d = np.zeros(n,dtype=float)
    d[idx_min] = 1

    sz = 2.0/(t+2)
    lbd = (1-sz) * lbd + sz * d

  return U.T.dot(lbd), lbd

In [ ]:
def find_sz(a,b):
  # find the minimizer of ax^2 + 2bx, where 0<=x<=1 (x is step size)

  # check that a is non negative
  if a < 0 :
    print("error in solving step size")
    return -1
  if a == 0:
    if b >=0 :
      sz = 0
    else:
      sz = 1
    return sz

  axis = - b * 1.0 / a
  if axis < 0 :
    sz = 0
  elif axis > 1:
    sz = 1
  else:
    sz = axis
  return sz

In [ ]:
# a FW procedure to solve for the min norm element
# with exact step size
def new_FW_solve_w(U, rounds, lambda_0=None, to_print=False):
  # with warm start of lbd
  n,d = U.shape
  if lambda_0 is None:
    lbd = 1.0/n * np.ones(n)
  else:
    lbd = lambda_0

  # precomputing G here is actually better since it can be reused in the loop,
  # otherwise matrix/matrix multiplication (UU^T) is not efficient
  G = np.array(U.dot(U.T))

  for t in range(rounds):
    v = G.dot(lbd)
    idx_min = np.argmin(v)
    d = np.zeros(n,dtype=float)
    d[idx_min] = 1

    # find the best sz by solving a quadratic problem
    delta = d - lbd
    a = delta.dot(G.dot(delta))
    b = delta.dot(v)
    sz = find_sz(a,b)
    if sz < 0:
      sys.exit("error in running FW for solving QP")

    lbd = (1-sz) * lbd + sz * d
    if sz == 0 and to_print:
      print("it takes ith round:",t)
      break

  return U.T.dot(lbd), lbd

## RP procedure

In [ ]:
def f(w, D_matrix):
    """
    Compute the function values and gradients for each f_i, where each f_i depends on
    the variables indicated in the input adjacency matrix D_matrix, and each f_i is a
    quadratic function of the variables it depends on.

    Parameters:
    w (np.ndarray): Input vector of variables (size d).
    D_matrix (np.ndarray): Dependency matrix (n x d), where D_matrix[i, j] = 1
                           indicates that function f_i depends on variable w_j.

    Returns:
    tuple: (f_val, grad_f)
           f_val: List of function values for each f_i (length n).
           grad_f: List of gradient vectors for each f_i (each vector is of length d).
    """
    d = w.size  # Number of variables (columns of D_matrix)
    n = D_matrix.shape[0]  # Number of functions (rows of D_matrix)

    f_val = [0] * n  # List to store function values
    grad_f = [0] * n  # List to store gradient vectors

    for i in range(n):
        D_row = D_matrix[i]  # Get the dependency row for f_i
        f_val[i] = 0
        grad_f[i] = np.zeros(d)

        for j in range(d):
            if D_row[j] != 0:  # If f_i depends on variable w_j
                # Compute the quadratic function value and gradient for w_j
                f_val[i] += (w[j] - i) ** 2
                grad_f[i][j] = 2 * (w[j] - i)

    return f_val, grad_f

In [ ]:
# Generate a random 20x100 dependency matrix
D_matrix_20x100 = generate_sparse_adj_matrix(20, 100, 0.1, seed=3)

# Create the bipartite graph
B, function_nodes, variable_nodes = adjacency_matrix_to_bipartite_graph(D_matrix_20x100)

# Run the iterative cycle detection and merging process
final_graph, merged_variable_list, function_nodes, variable_nodes = iterative_cycle_detection_and_merge(B, function_nodes, variable_nodes)


# Post-process the merged variable list to flatten any nested merges
flattened_merged_variable_list = post_process_merged_variables(merged_variable_list)

# Filter the merged variable list to only print those that exist in the final graph
existing_merged_variables = filter_existing_merged_nodes(flattened_merged_variable_list, final_graph)

# Create a dictionary mapping merged variables to their constituent variables
# Using only existing_merged_variables
merged_var_dict = {}
for merged_var_group in existing_merged_variables:
    merged_var = merged_var_group[0]
    constituent_vars = merged_var_group[1:]
    merged_var_dict[merged_var] = constituent_vars

# Update variable_nodes by replacing merged variables with their constituents
updated_variable_nodes = []
for var in variable_nodes:
    if var in merged_var_dict:
        updated_variable_nodes.extend(merged_var_dict[var])  # Replace with constituents
    else:
        updated_variable_nodes.append(var)  # Keep original variable

# Output the results
print("Final list of existing merged variable nodes:")
for merged_node, vars in existing_merged_variables:
    print(f"{merged_node}: merged from {vars}")

num_variable_nodes_final = len(variable_nodes)
print(f"Number of variable nodes in final graph: {num_variable_nodes_final}")


## MDGA

In [ ]:
# Initial values for w_0 (size 100)
scaling_factor = 1  # Adjust this value to control the scaling
seed = 100
np.random.seed(seed)
random.seed(seed)
w_0 = np.random.rand(100) * scaling_factor
w = w_0
norm_list = []

# Start measuring time
start_time = time.time()

# Main MGDA loop for the 20x100 matrix
for i in range(1000): # or 800
    # Compute function values and gradients for the current w and D_matrix
    f_val, grad_f = f(w, D_matrix_20x100)

    # Stack gradients into a matrix U
    U = np.vstack(grad_f)

    # Solve for the gradient direction using Frank-Wolfe
    grad, _ = new_FW_solve_w(U,rounds=10)

    # Record the norm of the gradient
    norm_list.append(np.linalg.norm(grad))

    # Update w using the step size sz
    sz = 0.01
    w = w - sz * grad

    # Print the logarithm of the norm of the gradient after 400 iterations
    if i == 799:
        print(np.linalg.norm(grad))
        print(w)  # Print the final value of w
        print(f(w, D_matrix_20x100)[0])  # Print the function values f(w)

# End measuring time
end_time = time.time()

# Calculate and print the runtime
runtime = end_time - start_time
print(f"Runtime: {runtime} seconds")


In [ ]:
store_w = w.copy()
store_func_values = f(w, D_matrix_20x100)[0].copy()

## RPMGDA

In [ ]:
variable_nodes = updated_variable_nodes.copy()
D_matrix = D_matrix_20x100.copy()

In [ ]:
# Function to convert string variable names ('w_i') to numerical indices
def var_name_to_index(var_name):
    """
    Convert a variable name of the form 'w_i' to an index (i-1), assuming 'i' starts from 1.
    """
    return int(var_name.split('_')[1]) - 1  # Convert 'w_i' to index i-1

In [ ]:
# Precompute the dependencies for each block
def precompute_dependencies(D_matrix, variable_nodes):
    """
    Precompute and store the dependent functions for each variable/block in variable_nodes.

    Returns:
    dependencies: List of tuples (block, dependent_functions, use_mgda)
                  where block is a list of variable indices, dependent_functions are the functions that depend on the block,
                  and use_mgda is a boolean indicating whether MGDA should be used.
    """
    dependencies = []

    for block in variable_nodes:
        if isinstance(block, list):
            # Convert block list from strings to indices
            block_indices = [var_name_to_index(var) for var in block]

            # Identify which functions depend on this block
            dependent_functions = []
            for j in range(D_matrix.shape[0]):  # Loop over functions
                if np.any(D_matrix[j, block_indices]):  # If any variable in block is dependent
                    dependent_functions.append(j)

            use_mgda = len(dependent_functions) >= 2
            dependencies.append((block_indices, dependent_functions, use_mgda))
        else:
            # Single variable, convert from string to index
            block_index = var_name_to_index(block)

            # Identify which functions depend on this single variable
            dependent_func_idx = np.where(D_matrix[:, block_index] == 1)[0]

            use_mgda = len(dependent_func_idx) >= 2
            dependencies.append(([block_index], dependent_func_idx, use_mgda))

    return dependencies

In [ ]:
# Precompute the dependencies outside of the loop
dependencies = precompute_dependencies(D_matrix, variable_nodes)

# RP-MGDA with same random initialization
w = w_0.copy()

# Use the following initialization if testing refinement of MGDA solution with RP-MGDA
# w = store_w.copy()

norm_list_RPMGDA = []

sz = 0.01  # Step size for gradient descent

# Start measuring time
start_time = time.time()

for i in range(1000):  # Running for 1000 iterations; postprocess with 5 iterations
    f_val, grad_f = f(w, D_matrix)  # Compute function values and gradients
    Jcb_f = np.stack(grad_f, axis=0)  # Jacobian of gradients
    norm_sq = 0

    # Use the precomputed dependencies to update blocks
    for block_indices, dependent_functions, use_mgda in dependencies:
        if use_mgda:
            # Use MGDA for blocks dependent on 2 or more functions
            U = Jcb_f[dependent_functions][:, block_indices]  # Extract rows corresponding to dependent functions
            grad, _ = new_FW_solve_w(U, rounds=10)  # Solve with MGDA step
            w[block_indices] = w[block_indices] - sz * grad  # Update block with MGDA step
            norm_sq += np.linalg.norm(grad) ** 2
        else:
            # Use gradient descent for blocks dependent on only one function
            dependent_func_idx = dependent_functions[0]
            grad_block = Jcb_f[dependent_func_idx, block_indices]
            w[block_indices] = w[block_indices] - sz * grad_block  # Gradient descent update
            norm_sq += np.linalg.norm(grad_block) ** 2


    # Concatenate block gradients and calculate the norm
    norm_list_RPMGDA.append(np.sqrt(norm_sq))


# End measuring time
end_time = time.time()

# Calculate and print the runtime
runtime = end_time - start_time
print(f"Runtime: {runtime} seconds")


# After the iterations, print final values
print("Final w:", w)
y_rpmgda = f(w, D_matrix)[0]
print("Final function values:", y_rpmgda)

In [ ]:
plt.figure(figsize=(10, 6))  # Adjust figure size for better visibility
plt.plot(norm_list_RPMGDA[:1000], linewidth=2, color='blue')  # Customize line properties
plt.plot(norm_list[:1000], linewidth=2, color='red')  # Customize line properties

plt.legend(['RPMGDA', 'MGDA'], fontsize=16)  # Add legend with font size
plt.xlabel("Iteration", fontsize=14)
plt.ylabel("Gradient Norm", fontsize=14)
plt.title("Gradient Norm vs. Iteration (RP-MGDA)", fontsize=18)

plt.grid(True, linestyle='--', alpha=0.7)  # Add a grid for better readability

plt.tight_layout()  # Adjust layout to prevent overlapping elements
plt.show()

## Investigate and save results

In [ ]:
# Find indices where y_rpmgda[i] >= store_func_values[i]
indices = [i for i, (a, b) in enumerate(zip(y_rpmgda, store_func_values)) if a >= b]

# Print the indices
print("Indices where y_rpmgda[i] >= store_func_values[i]:", indices)

In [ ]:
import pickle

# Assuming you have store_func_values and y_rpmgda already calculated

# Create a dictionary to store both variables
data_to_store = {
    'MGDA': store_func_values,
    'RPMGDA': y_rpmgda
}

# Save the dictionary to a pickle file
with open('matrix20_100_d_0p1_s3_scaling1_s100.pickle', 'wb') as file:
    pickle.dump(data_to_store, file)